# 🧱 `DIM_CURSO` (Gold) — Notebook pronto (lendo Silver e salvando em Gold)

Este notebook foi preparado para a estrutura:

```
/bronze
/silver
/gold
```

✅ Ele:
- lê **2018_anonimizado.xlsx** e **2019_anonimizado.xlsx** a partir de `../silver/`
- consolida em um único dataframe
- cria a dimensão **DIM_CURSO**
- preenche `AREA_GERAL_CINE` vazia como **"Não informado"**
- deduplica por `id_curso` mantendo o registro mais completo
- salva em `gold/output/dim_curso.csv` (ou seja, na mesma pasta do notebook, dentro de `output/`)

> **Como usar:** coloque este notebook dentro da pasta `gold/` do seu repositório e rode as células em ordem.


## 0) Imports e configuração de display

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## 1) Paths robustos (funciona com o notebook dentro de `gold/`)

In [ ]:

BASE_DIR = Path().resolve()  
OUT_DIR = BASE_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_FILES = [
    BASE_DIR / "2018_anonimizado.xlsx",
    BASE_DIR / "2019_anonimizado.xlsx",
]

print("📁 BASE_DIR:", BASE_DIR)
print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

print("📤 OUT_DIR:", OUT_DIR)


📁 BASE_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold
📥 INPUT_FILES:
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2018_anonimizado.xlsx | existe? True
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2019_anonimizado.xlsx | existe? True
📤 OUT_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output


## 2) Leitura dos arquivos (Silver) e consolidação

In [4]:
dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


✅ Linhas/Colunas consolidadas: (732261, 87)


,ULTIMO_PROCESSO,SITUACAO_DO_PROCESSO,IS_SEDE_EAD,NO_DO_PROCESSO,MODALIDADE,ANO_DO_PROTOCOLO,DATA,ORGAO,ATO,CATEGORIA_ATO,...,CINE_AREA_ESPECIFICA,CODIGO_AREA_GERAL_CINE,AREA_GERAL_CINE,CODIGO_AREA_DETALHADA_CINE,AREA_DETALHADA_CINE,CODIGO_AREA_ESPECIFICA_CINE,AREA_ESPECIFICA_CINE,ROTULO_CINE,AVALIACAO_OFICIAL,fonte_arquivo
0,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
1,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
2,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
3,NÃO,Aguardando Pagamento,S,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
4,NÃO,Arquivado,N,20070028,PRESENCIAL,2008,2008-09-26 00:00:00,SERES/DIREG/CGRERCES,Reconhecimento de Curso,Curso,...,Humanidades (exceto línguas),2,Artes e humanidades,223,Filosofia e ética,22,Humanidades (exceto línguas),Filosofia,Regulação,2018_anonimizado.xlsx


## 3) Funções auxiliares (padronização e deduplicação)

In [5]:
def norm_missing(s: pd.Series) -> pd.Series:
    """Padroniza nulos e strings vazias."""
    x = s.astype(str).str.strip()
    x = x.replace({
        "": np.nan,
        "nan": np.nan, "NAN": np.nan,
        "None": np.nan, "NONE": np.nan
    })
    return x

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    """Mantém a linha mais 'completa' por chave (mais colunas preenchidas)."""
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))


## 4) Validação de colunas do layout atual

In [ ]:
required = [
    "CODIGO_DO_CURSO",
    "NOME_CURSO_REGULACAO",   
    "GRAU",
    "CARGA_HORARIA_CADASTRO",
    "CODIGO_AREA_GERAL_CINE",
    "AREA_GERAL_CINE",
]

missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Colunas não encontradas no dataframe: {missing}")

print("✅ Colunas obrigatórias OK.")


✅ Colunas obrigatórias OK.


## 5) Construção da dimensão `DIM_CURSO`

In [7]:
dim_curso = df[[
    "CODIGO_DO_CURSO",
    "NOME_CURSO_REGULACAO",
    "GRAU",
    "CARGA_HORARIA_CADASTRO",
    "CODIGO_AREA_GERAL_CINE",
    "AREA_GERAL_CINE",
]].copy()

# Padronização
dim_curso["CODIGO_DO_CURSO"] = norm_missing(dim_curso["CODIGO_DO_CURSO"])
dim_curso["NOME_CURSO_REGULACAO"] = norm_missing(dim_curso["NOME_CURSO_REGULACAO"])
dim_curso["GRAU"] = norm_missing(dim_curso["GRAU"])
dim_curso["CARGA_HORARIA_CADASTRO"] = norm_missing(dim_curso["CARGA_HORARIA_CADASTRO"])
dim_curso["CODIGO_AREA_GERAL_CINE"] = norm_missing(dim_curso["CODIGO_AREA_GERAL_CINE"])
dim_curso["AREA_GERAL_CINE"] = norm_missing(dim_curso["AREA_GERAL_CINE"]).fillna("Não informado")

# Remover cursos sem id
dim_curso = dim_curso.dropna(subset=["CODIGO_DO_CURSO"])

# Renomear para padrão dimensional
dim_curso = dim_curso.rename(columns={
    "CODIGO_DO_CURSO": "id_curso",
    "NOME_CURSO_REGULACAO": "nome_curso",
    "GRAU": "grau",
    "CARGA_HORARIA_CADASTRO": "carga_horaria_cadastro",
    "CODIGO_AREA_GERAL_CINE": "cod_cine_area_geral",
    "AREA_GERAL_CINE": "cine_area_geral",
})

# Deduplicar por id_curso (mantendo a linha mais completa)
dim_curso = dedup_most_complete(dim_curso, "id_curso")

# Ordenar
dim_curso = dim_curso.sort_values("id_curso").reset_index(drop=True)

print("✅ DIM_CURSO pronta. Linhas:", len(dim_curso))
dim_curso.head(10)


✅ DIM_CURSO pronta. Linhas: 88437


,id_curso,nome_curso,grau,carga_horaria_cadastro,cod_cine_area_geral,cine_area_geral
0,1,DIREITO,Bacharelado,3904,4,"Negócios, administração e direito"
1,10,ENGENHARIA FLORESTAL,Bacharelado,4364,8,"Agricultura, silvicultura, pesca e veterinária"
2,10000,HISTÓRIA,Licenciatura,2800,1,Educação
3,100000,MARKETING,Tecnológico,1800,4,"Negócios, administração e direito"
4,100002,PRODUÇÃO SUCROALCOOLEIRA,Tecnológico,2600,7,"Engenharia, produção e construção"
5,1000030,MATEMÁTICA,Licenciatura,2840,1,Educação
6,100004,CURSO SUPERIOR DE TECNOLOGIA EM LOGÍSTICA,Tecnológico,1800,4,"Negócios, administração e direito"
7,100006,GESTÃO DE RECURSOS HUMANOS,Tecnológico,1800,4,"Negócios, administração e direito"
8,100008,GESTÃO DA PRODUÇÃO INDUSTRIAL,Tecnológico,2600,4,"Negócios, administração e direito"
9,100010,EDUCAÇÃO FÍSICA,Licenciatura,3200,1,Educação


## 6) Exportar CSV (Gold)

In [9]:
out_file = OUT_DIR / "dim_curso.csv"
dim_curso.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)

✅ Salvo em: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\dim_curso.csv
